### Data Pre-Processing

### Training Model

In [18]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.preprocessing.image import ImageDataGenerator, load_img, img_to_array
from tensorflow.keras.layers import Conv2D, Flatten, Dense, MaxPool2D, Dropout
from tensorflow.keras.models import Sequential
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split
import os
from sklearn.metrics import classification_report
import cv2

In [19]:
Directory = 'E:\Data\Face Mask Detection\Face-Mask-Detection-master\dataset'
Categories = ['with_mask', 'without_mask']
label=[i for i in range(len(Categories))]
label_dict=dict(zip(Categories,label))

In [20]:
label_dict

{'with_mask': 0, 'without_mask': 1}

In [21]:
data = []
labels = []
for category in Categories:
    path = os.path.join(Directory, category)
    for img in os.listdir(path):
        img_path = os.path.join(path, img)
        image = cv2.imread(img_path)
        try:
            gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
            resize_image = cv2.resize(gray, (100, 100))
            data.append(resize_image)
            labels.append(label_dict[category])
        except Exception as e:
            print('Exception', e)

Exception OpenCV(4.5.1) C:\Users\appveyor\AppData\Local\Temp\1\pip-req-build-kh7iq4w7\opencv\modules\imgproc\src\color.cpp:182: error: (-215:Assertion failed) !_src.empty() in function 'cv::cvtColor'

Exception OpenCV(4.5.1) C:\Users\appveyor\AppData\Local\Temp\1\pip-req-build-kh7iq4w7\opencv\modules\imgproc\src\color.cpp:182: error: (-215:Assertion failed) !_src.empty() in function 'cv::cvtColor'

Exception OpenCV(4.5.1) C:\Users\appveyor\AppData\Local\Temp\1\pip-req-build-kh7iq4w7\opencv\modules\imgproc\src\color.cpp:182: error: (-215:Assertion failed) !_src.empty() in function 'cv::cvtColor'

Exception OpenCV(4.5.1) C:\Users\appveyor\AppData\Local\Temp\1\pip-req-build-kh7iq4w7\opencv\modules\imgproc\src\color.cpp:182: error: (-215:Assertion failed) !_src.empty() in function 'cv::cvtColor'

Exception OpenCV(4.5.1) C:\Users\appveyor\AppData\Local\Temp\1\pip-req-build-kh7iq4w7\opencv\modules\imgproc\src\color.cpp:182: error: (-215:Assertion failed) !_src.empty() in function 'cv::cvtCol

In [22]:
data = np.array(data)/255
data = np.reshape(data, (data.shape[0], 100, 100, 1))
labels = np.array(labels)

In [23]:
#from tensorflow.keras.utils import np_utils
labels = keras.utils.to_categorical(labels)

In [24]:
labels

array([[1., 0.],
       [1., 0.],
       [1., 0.],
       ...,
       [0., 1.],
       [0., 1.],
       [0., 1.]], dtype=float32)

In [25]:
model = Sequential()

model.add(Conv2D(200, (3, 3), activation = 'relu', input_shape = data.shape[1:]))
model.add(MaxPool2D(pool_size = (2, 2)))
          
model.add(Conv2D(100, (3, 3), activation = 'relu'))
model.add(MaxPool2D(pool_size = (2, 2)))
          
model.add(Flatten())
model.add(Dropout(0.5))
          
model.add(Dense(50, activation = 'relu'))
          
model.add(Dense(2, activation = 'softmax'))
          
model.compile(loss = 'categorical_crossentropy', optimizer = 'adam', metrics = ['accuracy'])

In [26]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 98, 98, 200)       2000      
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 49, 49, 200)       0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 47, 47, 100)       180100    
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 23, 23, 100)       0         
_________________________________________________________________
flatten (Flatten)            (None, 52900)             0         
_________________________________________________________________
dropout (Dropout)            (None, 52900)             0         
_________________________________________________________________
dense (Dense)                (None, 50)                2

In [27]:
X_train, X_test, y_train, y_test = train_test_split(data, labels, test_size = 0.2, random_state = 0)

In [28]:
history = model.fit(X_train, y_train, validation_data = (X_test, y_test), epochs = 20)

Epoch 1/20
113/113 [==============================] - 107s 927ms/step - loss: 0.6544 - accuracy: 0.6251 - val_loss: 0.3771 - val_accuracy: 0.8451
Epoch 2/20
113/113 [==============================] - 105s 931ms/step - loss: 0.3835 - accuracy: 0.8350 - val_loss: 0.2626 - val_accuracy: 0.8905
Epoch 3/20
113/113 [==============================] - 104s 922ms/step - loss: 0.2625 - accuracy: 0.8942 - val_loss: 0.2818 - val_accuracy: 0.8761
Epoch 4/20
113/113 [==============================] - 109s 970ms/step - loss: 0.2087 - accuracy: 0.9219 - val_loss: 0.2478 - val_accuracy: 0.9038
Epoch 5/20
113/113 [==============================] - 106s 934ms/step - loss: 0.1932 - accuracy: 0.9205 - val_loss: 0.2091 - val_accuracy: 0.9192
Epoch 6/20
113/113 [==============================] - 108s 952ms/step - loss: 0.1194 - accuracy: 0.9598 - val_loss: 0.1917 - val_accuracy: 0.9226
Epoch 7/20
113/113 [==============================] - 109s 962ms/step - loss: 0.1203 - accuracy: 0.9529 - val_loss: 0.1719 -

In [3]:
from tensorflow.keras.models import load_model

In [29]:
import cv2

### Face Detection

In [33]:
face_clfr = cv2.CascadeClassifier('E:\pythonProject4\haarcascade_frontalface_default.xml')
cam = cv2.VideoCapture(0)
labels_dict={0:'MASK',1:'NO MASK'}
color_dict={0:(0,255,0),1:(0,0,255)}
while(True):

    ret,img=cam.read()
    gray=cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
    faces=face_clfr.detectMultiScale(gray,1.3,5)  

    for (x,y,w,h) in faces:
    
        face_img=gray[y:y+w,x:x+w]
        resized=cv2.resize(face_img,(100,100))
        normalized=resized/255.0
        reshaped=np.reshape(normalized,(1,100,100,1))
        result=model.predict(reshaped)

        
        label=np.argmax(result,axis=1)[0]
        label_1 = np.max(result,axis=1)[0]
        perc = 'Percentage is' + str(label_1*100) + '%'
        to_disp = '{}: {}'.format(labels_dict[label], perc)
        
        cv2.rectangle(img,(x,y),(x+w,y+h),color_dict[label],2)
        cv2.rectangle(img,(x,y-40),(x+w,y),color_dict[label],-1)
        #cv2.putText(img, labels_dict[label], (x, y-10),cv2.FONT_HERSHEY_SIMPLEX,0.8,(255,255,255),2)
        cv2.putText(img, to_disp, (x, y-10),cv2.FONT_HERSHEY_SIMPLEX,0.8,(255,255,255),2)
        
    cv2.imshow('LIVE',img)
    key=cv2.waitKey(1)
    
    if(key==13):
        break
        
cv2.destroyAllWindows()
cam.release()

In [31]:
cam.release()

In [34]:
os.chdir('E:\Data\Face Mask Detection')

In [35]:
model.save('Face_Mask.h5')

In [36]:
pip freeze> requirements.txt

Note: you may need to restart the kernel to use updated packages.
